In [1]:
# Importing dependancies and data from weather_py output csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pygeocoder import Geocoder

# Import API key
from config import g_key

gmaps.configure(api_key=g_key)

csv_path = "../weather_py/output_data/weather_csv"

weather_data = pd.read_csv(csv_path)
weather_data.dropna()

,city,country,lat,lng,max_temp (f),humidity (%),cloudiness (%),wind_speed (mph),date
0,aksehir,tr,38.3575,31.4164,32.738,84,1,1.69,2021-03-01
1,hilo,us,19.7297,-155.0900,77.000,65,75,7.20,2021-03-01
2,tomari,ru,47.7658,142.0683,12.542,92,100,2.99,2021-03-01
3,khatanga,ru,71.9667,102.5000,-32.116,85,96,1.72,2021-03-01
4,nanakuli,us,21.3906,-158.1547,78.800,50,1,6.69,2021-03-01
...,...,...,...,...,...,...,...,...,...
547,puga,mx,21.5833,-104.8167,86.000,37,40,5.14,2021-03-01
548,khani,ru,41.9563,42.9566,35.600,93,75,1.54,2021-03-01
549,airai,pw,-8.9266,125.4092,59.180,100,81,0.76,2021-03-01
550,matay,eg,28.4190,30.7792,53.942,63,0,4.83,2021-03-01


In [2]:
location = weather_data[["lat", "lng"]].astype(float)
humidity = weather_data["humidity (%)"].astype(float)

In [3]:
# create gmaps heatmap layer with points in each city of the weather datframe weighted by humidity 
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                    dissipating=False, max_intensity=100, point_radius = 1)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Selecting for ideal conditions, personalized to indivual's specifications to create short list of travel destinations
best_cities = weather_data[weather_data["max_temp (f)"] <= 90]
best_cities = best_cities[best_cities["max_temp (f)"] >= 75]
best_cities = best_cities[best_cities["humidity (%)"] <= 80]
best_cities = best_cities[best_cities["humidity (%)"] >= 40]
best_cities = best_cities[best_cities["cloudiness (%)"] <= 20]
best_cities = best_cities[best_cities["wind_speed (mph)"] < 4]
best_cities

,city,country,lat,lng,max_temp (f),humidity (%),cloudiness (%),wind_speed (mph),date
69,acapulco,mx,16.8634,-99.8901,82.400,69,1,3.60,2021-03-01
75,rikitea,pf,-23.1203,-134.9692,78.458,62,6,2.43,2021-03-01
101,saint-philippe,re,-21.3585,55.7679,75.200,64,0,2.57,2021-03-01
173,pisco,pe,-13.7000,-76.2167,82.400,61,0,3.09,2021-03-01
189,kununurra,au,-15.7667,128.7333,77.000,70,4,2.06,2021-03-01
223,port macquarie,au,-31.4333,152.9167,75.992,72,5,1.34,2021-03-01
250,grand gaube,mu,-20.0064,57.6608,75.992,79,10,2.24,2021-03-01
324,saint-leu,re,-21.1500,55.2833,75.002,58,1,0.58,2021-03-01
337,vallenar,cl,-28.5708,-70.7581,77.792,44,0,3.19,2021-03-01
341,cap malheureux,mu,-19.9842,57.6142,75.992,79,1,3.13,2021-03-01


In [5]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Setting search parameters 
search_type = "hotel"
radius = 5000
hotel_names = []
hotel_lats = []
hotel_lngs = []

params = {
   
    "radius": radius,
    "type": search_type,
    "key": g_key
}

# Iterate through best_cities setting lat/lng as location parameter 
for index, row in best_cities.iterrows():
    
    try:
       
        lat = row["lat"]
        lng = row["lng"]
        city = row["city"]
        
        params["location"] = f"{lat},{lng}"

        search = requests.get(base_url, params=params)
        search_result = search.json()
        
        # Selecting the second hotel from the json object and getting the name and location of the hotel
        hotel_name = search_result["results"][1]["name"]
        hotel_lat = search_result["results"][1]["geometry"]["location"]["lat"] 
        hotel_lng = search_result["results"][1]["geometry"]["location"]["lng"]
        
        # Append to list
        hotel_names.append(hotel_name)
        hotel_lats.append(hotel_lat) 
        hotel_lngs.append(hotel_lng)
        
        print(f"found {hotel_name} in {city}")
              
    except:
              
        print("oops. no hotel found") 

found HS Hotsson Smart Acapulco in acapulco
found Pension Maro'i in rikitea
found Chambres d'hôte "La Trinité" in saint-philippe
found Hotel San Isidro Oficial in pisco
found Hotel Kununurra in kununurra
found Rydges Port Macquarie in port macquarie
found Veranda Paul et Virginie Hotel & Spa in grand gaube
found Église du Sacré-Cœur de Saint-Leu in saint-leu
found Humacao Bed & Breakfast in vallenar
found Pereybere Hotel & Spa in cap malheureux
found All India Radio Kavaratti in kavaratti
found Solo Deportes in moron
found Peace Land Hotel in ifo
found Quality Suites Vila Velha in vila velha
found Trou aux Biches Beachcomber Golf Resort & Spa in grand baie


In [6]:
# Adding hotel information to best_cities
best_cities["hotel"] = hotel_names
best_cities["hotel_lat"] = hotel_lats
best_cities["hotel_lng"] = hotel_lngs
best_cities

,city,country,lat,lng,max_temp (f),humidity (%),cloudiness (%),wind_speed (mph),date,hotel,hotel_lat,hotel_lng
69,acapulco,mx,16.8634,-99.8901,82.400,69,1,3.60,2021-03-01,HS Hotsson Smart Acapulco,16.860816,-99.876302
75,rikitea,pf,-23.1203,-134.9692,78.458,62,6,2.43,2021-03-01,Pension Maro'i,-23.120620,-134.968753
101,saint-philippe,re,-21.3585,55.7679,75.200,64,0,2.57,2021-03-01,"Chambres d'hôte ""La Trinité""",-21.358434,55.770322
173,pisco,pe,-13.7000,-76.2167,82.400,61,0,3.09,2021-03-01,Hotel San Isidro Oficial,-13.707869,-76.198990
189,kununurra,au,-15.7667,128.7333,77.000,70,4,2.06,2021-03-01,Hotel Kununurra,-15.776818,128.738663
223,port macquarie,au,-31.4333,152.9167,75.992,72,5,1.34,2021-03-01,Rydges Port Macquarie,-31.428408,152.909174
250,grand gaube,mu,-20.0064,57.6608,75.992,79,10,2.24,2021-03-01,Veranda Paul et Virginie Hotel & Spa,-20.006569,57.668077
324,saint-leu,re,-21.1500,55.2833,75.002,58,1,0.58,2021-03-01,Église du Sacré-Cœur de Saint-Leu,-21.137612,55.294399
337,vallenar,cl,-28.5708,-70.7581,77.792,44,0,3.19,2021-03-01,Humacao Bed & Breakfast,-28.573242,-70.759710
341,cap malheureux,mu,-19.9842,57.6142,75.992,79,1,3.13,2021-03-01,Pereybere Hotel & Spa,-19.994261,57.591778


In [7]:
# Adding the hotel marker layer to the heatmap with pins
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in best_cities.iterrows()]
locations = best_cities[["hotel_lat", "hotel_lng"]]

markers = gmaps.marker_layer(locations)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))